In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的OpenAI API环境变量
load_dotenv()

True

# 一、通过Langchain中的HypotheticalDocumentEmbedder来实现HyDE

In [2]:
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [3]:
# 测试数据
documents = [
    Document(page_content="Python 是一种高级编程语言，广泛用于数据科学和机器学习。"),
    Document(page_content="机器学习是人工智能的一个分支，专注于算法和统计模型。"),
    Document(page_content="深度学习是机器学习的一个子领域，使用神经网络进行建模。"),
]
query = "Python 在数据科学中的应用"

In [4]:
# 初始化基础嵌入模型
base_embeddings = OpenAIEmbeddings()
# 初始化语言模型
llm = OpenAI()

# 使用web_search提示加载HyDE
embeddings = HypotheticalDocumentEmbedder.from_llm(llm, base_embeddings, "web_search")

# 现在我们可以像使用任何嵌入类一样使用它
result = embeddings.embed_query(query)
result

[-0.009304807521402836,
 -0.0037448573857545853,
 0.0064249043352901936,
 -0.025843774899840355,
 -0.012509068474173546,
 -0.00048162208986468613,
 -0.01390479039400816,
 -0.0118734585121274,
 -0.0011991406790912151,
 -0.0372716523706913,
 0.003551553236320615,
 0.01820334978401661,
 -0.011624456383287907,
 -0.003433604957535863,
 -0.02155832201242447,
 0.004554113373160362,
 0.009652099572122097,
 -0.01302017830312252,
 -0.0021509176585823298,
 0.015542960725724697,
 -0.01748255454003811,
 0.02052299864590168,
 -0.0021525558549910784,
 -0.013046388514339924,
 -0.027704736217856407,
 0.041544001549482346,
 0.003551553236320615,
 -0.026708729565143585,
 -0.02050989307463169,
 -0.0022492078132927418,
 0.01774466037750244,
 -0.009206517599523067,
 -0.006736156530678272,
 -0.01864893175661564,
 -0.020405050367116928,
 -0.004829326178878546,
 -0.020182259380817413,
 -0.007542136125266552,
 0.010680871084332466,
 -0.002188595477491617,
 0.042880747467279434,
 0.010235288180410862,
 -0.006985

In [5]:
docsearch1 = FAISS.from_documents(documents, embeddings)
similar_docs = docsearch1.similarity_search(query, k=2)
similar_docs

[Document(id='fd3b95ef-5886-475c-a5d0-88846cef8f38', metadata={}, page_content='Python 是一种高级编程语言，广泛用于数据科学和机器学习。'),
 Document(id='0907f620-fbbf-41fb-b178-e6748f7886d1', metadata={}, page_content='机器学习是人工智能的一个分支，专注于算法和统计模型。')]

## 生成多文档

In [6]:
# 初始化返回多个结果的语言模型
multi_llm = OpenAI(n=4, best_of=4)

# 使用多生成语言模型加载HyDE
multi_embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, base_embeddings, "web_search"
)

# 嵌入查询
multi_result = multi_embeddings.embed_query("Python 在数据科学中的应用")
multi_result

[0.001332574407570064,
 -0.0007643536664545536,
 0.015199903398752213,
 -0.02877262979745865,
 -0.01935824565589428,
 0.00014518279931508005,
 -0.011054474860429764,
 -0.004303625784814358,
 0.005604722071439028,
 -0.03398992866277695,
 -0.002929887967184186,
 0.0176923256367445,
 0.0003410131321288645,
 -0.004729791544377804,
 -0.004758848343044519,
 0.0060502588748931885,
 0.009330442175269127,
 -0.022251006215810776,
 -0.0037321767304092646,
 0.017911866307258606,
 -0.02203146554529667,
 0.018041007220745087,
 0.0002110649220412597,
 0.0006743585108779371,
 -0.02349076047539711,
 0.027455389499664307,
 0.0032139981631189585,
 -0.026473918929696083,
 -0.020068520680069923,
 -0.0028475604485720396,
 0.020249318331480026,
 -0.013908492401242256,
 -0.007509552873671055,
 -0.017911866307258606,
 -0.026047753170132637,
 -0.002700662473216653,
 -0.014037633314728737,
 -0.004771762527525425,
 0.009601637721061707,
 -0.0033382964320480824,
 0.04680072143673897,
 0.016581712290644646,
 -0.005

In [7]:
docsearch = FAISS.from_documents(documents, multi_embeddings)

In [8]:
similar_docs = docsearch.similarity_search(query, k=2)
similar_docs

[Document(id='a5fddf5a-64e1-4e73-889e-cdfee06f24ab', metadata={}, page_content='Python 是一种高级编程语言，广泛用于数据科学和机器学习。'),
 Document(id='1c0eac49-6c7c-4377-a285-605ac287c174', metadata={}, page_content='机器学习是人工智能的一个分支，专注于算法和统计模型。')]

# 二、通过自定义Prompt来实现HyDE

In [9]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# 1. 定义生成假设性文档的 Prompt
hyde_prompt = PromptTemplate(
    input_variables=["query"],
    template="请根据以下问题生成一个假设性文档，描述可能的答案内容：\n问题：{query}\n假设性文档："
)

# 2. 初始化 LLM 和 Embeddings
llm = OpenAI()  # 用于生成假设性文档
embeddings = OpenAIEmbeddings()  # 用于文档嵌入

# 3. 构建生成假设性文档的链
hyde_chain = hyde_prompt | llm

# 4. 定义检索器（假设已有文档库）
documents = [
    Document(page_content="Python 是一种高级编程语言，广泛用于数据科学和机器学习。"),
    Document(page_content="机器学习是人工智能的一个分支，专注于算法和统计模型。"),
    Document(page_content="深度学习是机器学习的一个子领域，使用神经网络进行建模。"),
]
vectorstore = FAISS.from_documents(documents, embeddings)  # 构建向量存储
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # 检索 Top-3 文档

# 5. 实现 HyDE 流程
def hyde_retrieval(query):
    # 生成假设性文档
    hypothetical_document = hyde_chain.invoke(query)
    print(f"生成的假设性文档：{hypothetical_document}")

    # 使用假设性文档进行检索
    relevant_docs = retriever.invoke(hypothetical_document)
    print(f"检索到的相关文档：{relevant_docs}")

    return relevant_docs

# 6. 测试 HyDE
query = "Python 在数据科学中的应用"
relevant_docs = hyde_retrieval(query)

生成的假设性文档：
介绍：
Python 是一种高级、通用的编程语言，具有简单易学、功能强大的特点，因此在数据科学领域得到了广泛的应用。本文将介绍Python在数据科学中的应用，包括数据处理、数据分析、机器学习和人工智能等方面。

数据处理：
Python拥有丰富的数据处理库，如NumPy、Pandas等，可以快速高效地处理各种类型的数据。它提供了灵活的数据结构和强大的数据操作功能，可以轻松实现数据清洗、合并、转换等操作。此外，Python还可以与其他工具和数据库进行无缝集成，使得数据处理更加便捷。

数据分析：
Python在数据分析领域也有着出色的表现。它结合了强大的数据处理能力和丰富的可视化工具，可以
检索到的相关文档：[Document(id='5447beae-3690-412e-9889-533b9677e15e', metadata={}, page_content='Python 是一种高级编程语言，广泛用于数据科学和机器学习。'), Document(id='e43da8a8-c614-46a0-b5ea-89a042efa45a', metadata={}, page_content='机器学习是人工智能的一个分支，专注于算法和统计模型。')]
